In [ ]:
!pip install pykalman
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.3 MB/s eta 0:00:00


## 더미변수 포함(Random Effect와 Fixed Effect 동시에 확인)

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-25-58ee1529768c>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_Acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 변수 생성
if 'Language_Game_Flag' not in data.columns:
    data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 전체 데이터를 그대로 사용 (샘플링 제거)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'similarity_score',
                     'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'day__games', 'Category_ID_youtube',
                    'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day_unsimilar_youtube', 'video_potentiality_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# 변수명에서 '+' 기호를 '_'로 교체하여 에러 방지
independent_vars = [var.replace('+', '_') for var in independent_vars]
data.columns = data.columns.str.replace('+', '_')

# VIF 계산을 위한 데이터 준비
X = data[independent_vars].select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Games_Flag와 Language_Game_Flag를 명시적으로 추가
for flag in ['Games_Flag', 'Language_Game_Flag']:
    if flag not in low_vif_features:
        low_vif_features.append(flag)

# Kalman Filter 설정
user_acquisition = data['User_Acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 날짜, AppName, Country를 패널 분석용 식별자로 사용 (2개의 계층만 사용)
data['Date'] = pd.to_datetime(data['Date'])

# MultiIndex 설정 (AppName과 Date 또는 Country와 Date)
panel_data = data.set_index(['AppName_original', 'Date'])

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = PanelOLS.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm)

# 랜덤효과 패널 회귀 분석 (Permanent Component 분석)
model_perm_re = RandomEffects.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Permanent Component):")
print(model_perm_re)

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = PanelOLS.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp)

# 랜덤효과 패널 회귀 분석 (Temporary Component 분석)
model_temp_re = RandomEffects.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Temporary Component):")
print(model_temp_re)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


고정효과 패널 분석 결과 (Permanent Component):
                           PanelOLS Estimation Summary                           
Dep. Variable:     Permanent_Component   R-squared:                        0.7302
Estimator:                    PanelOLS   R-squared (Between):              0.5440
No. Observations:               291809   R-squared (Within):               0.7302
Date:                 Thu, Oct 24 2024   R-squared (Overall):              0.6127
Time:                         13:34:13   Log-likelihood                -1.709e+06
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   4.937e+04
Entities:                           13   P-value                           0.0000
Avg Obs:                     2.245e+04   Distribution:               F(16,291780)
Min Obs:                     1.017e+04                                           
Max Obs:                     3.318e+04   F-statistic (robust)

## VIF TEST 제외

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')
data

<ipython-input-172-4b3032fe82e6>:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,Bound,...,video_potentiality_games,day+_youtube,day+_games,Category_ID_youtube,Category_ID_games,similarity_score_games,Daily Average Rating,Total Average Rating,View_Count_previous_games,video_potentiality_previous_games
0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,40,1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
1,2021-07-29,퍼즐놀이,Australia,오세아니아,영어,12,1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
2,2021-07-29,퍼즐놀이,Austria,유럽,영어,1,0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
3,2021-07-29,퍼즐놀이,Brazil,중남미,포르투갈어,102,10,0.0,0.0,2.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
4,2021-07-29,퍼즐놀이,Canada,북미,영어,21,3,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291804,2024-08-09,여름휴가,Taiwan,아시아,중국어,23,8,0.0,0.0,24.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,2.8750,0.0,0.0
291805,2024-08-09,여름휴가,United Arab Emirates,중동,아랍어,33,23,0.0,818.0,26.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,3.8571,0.0,0.0
291806,2024-08-09,여름휴가,United Kingdom,유럽,영어,72,44,0.0,1920.0,51.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,4.0000,0.0,0.0
291807,2024-08-09,여름휴가,United States,북미,영어,378,218,793.0,11026.0,314.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,3.5416,0.0,0.0


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')
# 국가와 앱 이름별로 그룹화하여 그룹별 평균값으로 결측치 채우기
grouped = data.groupby(['Country', 'AppName'])

# 그룹의 평균값으로 결측치 채우기
data['Daily Average Rating'] = grouped['Daily Average Rating'].transform(lambda x: x.fillna(x.mean()))
data['Total_Average_Rating'] = grouped['Total_Average_Rating'].transform(lambda x: x.fillna(x.mean()))

# 만약 그룹에 평균값도 없다면, 전체 평균으로 한 번 더 채워주는 코드 추가
data['Daily Average Rating'].fillna(data['Daily Average Rating'].mean(), inplace=True)
data['Total_Average_Rating'].fillna(data['Total_Average_Rating'].mean(), inplace=True)
data.drop(columns = 'Daily Average Rating',axis = 1, inplace = True)


<ipython-input-188-7a965b41b95e>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')
<ipython-input-188-7a965b41b95e>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Daily Average Rating'].fillna(data['Daily Average Rating'].mean(), inplace=True)
<ipython-input-188-7a965b41b95e>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through ch

In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools.tools import add_constant
from pykalman import KalmanFilter

# 불필요한 'Unnamed: 0' 열 제거
data = data.drop(columns=['Unnamed: 0'])

# Date 변수를 datetime으로 변환
data['Date'] = pd.to_datetime(data['Date'])

# 문자형 데이터인 'CTR'과 'Win rate'를 숫자형으로 변환 (예시)
data['CTR'] = pd.to_numeric(data['CTR'], errors='coerce')
data['Win_rate'] = pd.to_numeric(data['Win rate'], errors='coerce')

# 범주형 변수를 더미 변수로 변환 (Country, AppName, CONTINENT, AD_LAN)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# 새로운 YouTube_Flag 생성: unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('non_similar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# Kalman 필터 적용 함수 (Country_original과 Date별로 적용)
def apply_kalman_filter(group):
    user_acquisition = group['User_Acquisition'].values

    # Kalman Filter 초기화
    kf = KalmanFilter(
        transition_matrices=np.array([[1, 0.1], [0, 0.9]]),  # 전이 행렬
        observation_matrices=np.array([[1, 1]]),  # 관찰 행렬
        initial_state_mean=[user_acquisition[0], 0]  # 초기 상태값
    )

    # Kalman Filter 적용
    state_means, _ = kf.filter(user_acquisition)

    # Permanent 및 Temporary Component 추출
    group['Permanent_Component'] = state_means[:, 0]
    group['Temporary_Component'] = state_means[:, 1]
    return group

# Country_original과 날짜별로 그룹화하여 Kalman 필터 적용
grouped_data = data.groupby(['Country_original', 'Date']).apply(apply_kalman_filter)

# 상수항 추가 (Intercept)
grouped_data = add_constant(grouped_data)

# 사용할 독립 변수들 선택
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'Category_ID_youtube',
                    'Category_ID_games', 'Total_Average_Rating', 'View_Count_previous_games',
                    'YouTube_Flag', 'Games_Flag']

# 패널 데이터를 위한 독립 변수만 선택
panel_data = grouped_data[['Date', 'Permanent_Component', 'Temporary_Component'] + independent_vars]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')

# 패널 데이터를 MultiIndex로 변환 (Country_original과 Date를 기준으로)
panel_data = panel_data.set_index(['Country_original', 'Date'])

# 독립 변수와 종속 변수 분리
X_panel = panel_data[independent_vars]
y_panel_perm = panel_data['Permanent_Component']
y_panel_temp = panel_data['Temporary_Component']

# PanelOLS 적용 (Permanent Component 분석)
model_perm = PanelOLS(y_panel_perm, X_panel, entity_effects=True, time_effects=True).fit()

# 결과 출력 (Permanent Component)
print("Panel OLS 분석 결과 (Permanent Component):")
print(model_perm.summary)

# PanelOLS 적용 (Temporary Component 분석)
model_temp = PanelOLS(y_panel_temp, X_panel, entity_effects=True, time_effects=True).fit()

# 결과 출력 (Temporary Component)
print("\nPanel OLS 분석 결과 (Temporary Component):")
print(model_temp.summary)

KeyError: 'Country_original'

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291809 entries, 0 to 291808
Data columns (total 94 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   Unnamed: 0                             291809 non-null  int64         
 1   Date                                   291809 non-null  datetime64[ns]
 2   Store_Visit                            291809 non-null  int64         
 3   Store_Acquisition                      291809 non-null  int64         
 4   DAU                                    291809 non-null  float64       
 5   MAU                                    291809 non-null  float64       
 6   Bound                                  291809 non-null  float64       
 7   User_Acquisition                       291809 non-null  float64       
 8   Impressions                            291809 non-null  float64       
 9   Est. earnings (KRW)                    291809 no

## Hausman Test

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-29-58ee1529768c>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects, compare
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_Acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 변수 생성
if 'Language_Game_Flag' not in data.columns:
    data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 전체 데이터를 그대로 사용 (샘플링 제거)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'similarity_score',
                     'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'day__games', 'Category_ID_youtube',
                    'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day_unsimilar_youtube', 'video_potentiality_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# 변수명에서 '+' 기호를 '_'로 교체하여 에러 방지
independent_vars = [var.replace('+', '_') for var in independent_vars]
data.columns = data.columns.str.replace('+', '_')

# VIF 계산을 위한 데이터 준비
X = data[independent_vars].select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Games_Flag와 Language_Game_Flag를 명시적으로 추가
for flag in ['Games_Flag', 'Language_Game_Flag']:
    if flag not in low_vif_features:
        low_vif_features.append(flag)

# Kalman Filter 설정
user_acquisition = data['User_Acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 날짜, AppName, Country를 패널 분석용 식별자로 사용 (2개의 계층만 사용)
data['Date'] = pd.to_datetime(data['Date'])

# MultiIndex 설정 (AppName과 Date 또는 Country와 Date)
panel_data = data.set_index(['AppName_original', 'Date'])

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = PanelOLS.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm)

# 랜덤효과 패널 회귀 분석 (Permanent Component 분석)
model_perm_re = RandomEffects.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Permanent Component):")
print(model_perm_re)

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = PanelOLS.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp)

# 랜덤효과 패널 회귀 분석 (Temporary Component 분석)
model_temp_re = RandomEffects.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Temporary Component):")
print(model_temp_re)

# Hausman 테스트 수행 (Permanent Component)
hausman_test_perm = compare({'Fixed Effects': model_perm, 'Random Effects': model_perm_re})

# Hausman 테스트 결과 출력 (Permanent Component)
print("\nHausman 테스트 결과 (Permanent Component):")
print(hausman_test_perm)

# Hausman 테스트 수행 (Temporary Component)
hausman_test_temp = compare({'Fixed Effects': model_temp, 'Random Effects': model_temp_re})

# Hausman 테스트 결과 출력 (Temporary Component)
print("\nHausman 테스트 결과 (Temporary Component):")
print(hausman_test_temp)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


고정효과 패널 분석 결과 (Permanent Component):
                           PanelOLS Estimation Summary                           
Dep. Variable:     Permanent_Component   R-squared:                        0.7302
Estimator:                    PanelOLS   R-squared (Between):              0.5440
No. Observations:               291809   R-squared (Within):               0.7302
Date:                 Thu, Oct 24 2024   R-squared (Overall):              0.6127
Time:                         15:30:03   Log-likelihood                -1.709e+06
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   4.937e+04
Entities:                           13   P-value                           0.0000
Avg Obs:                     2.245e+04   Distribution:               F(16,291780)
Min Obs:                     1.017e+04                                           
Max Obs:                     3.318e+04   F-statistic (robust)

### Hausman Test P-value 확인

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-34-58ee1529768c>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter
from scipy.stats import chi2

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_Acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 변수 생성
if 'Language_Game_Flag' not in data.columns:
    data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 전체 데이터를 그대로 사용 (샘플링 제거)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'similarity_score',
                     'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'day__games', 'Category_ID_youtube',
                    'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day_unsimilar_youtube', 'video_potentiality_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# 변수명에서 '+' 기호를 '_'로 교체하여 에러 방지
independent_vars = [var.replace('+', '_') for var in independent_vars]
data.columns = data.columns.str.replace('+', '_')

# VIF 계산을 위한 데이터 준비
X = data[independent_vars].select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Games_Flag와 Language_Game_Flag를 명시적으로 추가
for flag in ['Games_Flag', 'Language_Game_Flag']:
    if flag not in low_vif_features:
        low_vif_features.append(flag)

# Kalman Filter 설정
user_acquisition = data['User_Acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 날짜, AppName, Country를 패널 분석용 식별자로 사용 (2개의 계층만 사용)
data['Date'] = pd.to_datetime(data['Date'])

# MultiIndex 설정 (AppName과 Date 또는 Country와 Date)
panel_data = data.set_index(['AppName_original', 'Date'])

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = PanelOLS.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm)

# 랜덤효과 패널 회귀 분석 (Permanent Component 분석)
model_perm_re = RandomEffects.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Permanent Component):")
print(model_perm_re)

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = PanelOLS.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp)

# 랜덤효과 패널 회귀 분석 (Temporary Component 분석)
model_temp_re = RandomEffects.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Temporary Component):")
print(model_temp_re)

# Hausman 테스트 수동 계산
beta_fixed_perm = model_perm.params.values
beta_random_perm = model_perm_re.params.values
cov_fixed_perm = model_perm.cov.values
cov_random_perm = model_perm_re.cov.values

# 차이 계산
diff_perm = beta_fixed_perm - beta_random_perm
cov_diff_perm = cov_fixed_perm - cov_random_perm

# Wald 통계량 계산
wald_stat_perm = np.dot(np.dot(diff_perm.T, np.linalg.inv(cov_diff_perm)), diff_perm)

# p-value 계산 (자유도는 차이의 차원)
df_perm = len(diff_perm)
p_value_perm = chi2.sf(wald_stat_perm, df_perm)

print(f"\nPermanent Component - Hausman Test p-value: {p_value_perm}")

# Temporary Component에 대해서도 동일하게 수행
beta_fixed_temp = model_temp.params.values
beta_random_temp = model_temp_re.params.values
cov_fixed_temp = model_temp.cov.values
cov_random_temp = model_temp_re.cov.values

# 차이 계산
diff_temp = beta_fixed_temp - beta_random_temp
cov_diff_temp = cov_fixed_temp - cov_random_temp

# Wald 통계량 계산
wald_stat_temp = np.dot(np.dot(diff_temp.T, np.linalg.inv(cov_diff_temp)), diff_temp)

# p-value 계산 (자유도는 차이의 차원)
df_temp = len(diff_temp)
p_value_temp = chi2.sf(wald_stat_temp, df_temp)

print(f"\nTemporary Component - Hausman Test p-value: {p_value_temp}")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


고정효과 패널 분석 결과 (Permanent Component):
                           PanelOLS Estimation Summary                           
Dep. Variable:     Permanent_Component   R-squared:                        0.7302
Estimator:                    PanelOLS   R-squared (Between):              0.5440
No. Observations:               291809   R-squared (Within):               0.7302
Date:                 Thu, Oct 24 2024   R-squared (Overall):              0.6127
Time:                         15:45:48   Log-likelihood                -1.709e+06
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   4.937e+04
Entities:                           13   P-value                           0.0000
Avg Obs:                     2.245e+04   Distribution:               F(16,291780)
Min Obs:                     1.017e+04                                           
Max Obs:                     3.318e+04   F-statistic (robust)

### Segment 요소 추가

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-4-58ee1529768c>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [ ]:
segment = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/segment.csv')

In [ ]:
segment_cocobi = pd.merge(data, segment, on = 'Country', how = 'left')

In [ ]:
data = segment_cocobi.copy()

In [ ]:
data['segment'] = data['segment'].astype(str)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291809 entries, 0 to 291808
Data columns (total 40 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Unnamed: 0                             291809 non-null  int64  
 1   Date                                   291809 non-null  object 
 2   AppName                                291809 non-null  object 
 3   Country                                291809 non-null  object 
 4   CONTINENT                              291809 non-null  object 
 5   AD_LAN                                 291809 non-null  object 
 6   Store_Visit                            291809 non-null  int64  
 7   Store_Acquisition                      291809 non-null  int64  
 8   DAU                                    291809 non-null  float64
 9   MAU                                    291809 non-null  float64
 10  Bound                                  291809 non-null  

## Country 대신 Segment를 넣고 해당 변수를 더미변수로 지정

In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter
from scipy.stats import chi2

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_Acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']
data['segment_original'] = data['segment']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country, AppName, AD_LAN, segment 등)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'AD_LAN', 'segment'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 변수 생성
if 'Language_Game_Flag' not in data.columns:
    data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 전체 데이터를 그대로 사용 (샘플링 제거)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'similarity_score',
                     'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'day__games', 'Category_ID_youtube',
                    'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day_unsimilar_youtube', 'video_potentiality_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# 변수명에서 '+' 기호를 '_'로 교체하여 에러 방지
independent_vars = [var.replace('+', '_') for var in independent_vars]
data.columns = data.columns.str.replace('+', '_')

# VIF 계산을 위한 데이터 준비
X = data[independent_vars].select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Games_Flag와 Language_Game_Flag를 명시적으로 추가
for flag in ['Games_Flag', 'Language_Game_Flag']:
    if flag not in low_vif_features:
        low_vif_features.append(flag)

# Kalman Filter 설정
user_acquisition = data['User_Acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 날짜, AppName, Country를 패널 분석용 식별자로 사용 (2개의 계층만 사용)
data['Date'] = pd.to_datetime(data['Date'])

# MultiIndex 설정 (AppName과 Date 또는 Country와 Date)
panel_data = data.set_index(['AppName_original', 'Date'])

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = PanelOLS.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm)

# 랜덤효과 패널 회귀 분석 (Permanent Component 분석)
model_perm_re = RandomEffects.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Permanent Component):")
print(model_perm_re)

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = PanelOLS.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + EntityEffects', data=panel_data).fit()

# 고정효과 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp)

# 랜덤효과 패널 회귀 분석 (Temporary Component 분석)
model_temp_re = RandomEffects.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Temporary Component):")
print(model_temp_re)

# Hausman 테스트 수동 계산
beta_fixed_perm = model_perm.params.values
beta_random_perm = model_perm_re.params.values
cov_fixed_perm = model_perm.cov.values
cov_random_perm = model_perm_re.cov.values

# 차이 계산
diff_perm = beta_fixed_perm - beta_random_perm
cov_diff_perm = cov_fixed_perm - cov_random_perm

# Wald 통계량 계산
wald_stat_perm = np.dot(np.dot(diff_perm.T, np.linalg.inv(cov_diff_perm)), diff_perm)

# p-value 계산 (자유도는 차이의 차원)
df_perm = len(diff_perm)
p_value_perm = chi2.sf(wald_stat_perm, df_perm)

print(f"\nPermanent Component - Hausman Test p-value: {p_value_perm}")

# Temporary Component에 대해서도 동일하게 수행
beta_fixed_temp = model_temp.params.values
beta_random_temp = model_temp_re.params.values
cov_fixed_temp = model_temp.cov.values
cov_random_temp = model_temp_re.cov.values

# 차이 계산
diff_temp = beta_fixed_temp - beta_random_temp
cov_diff_temp = cov_fixed_temp - cov_random_temp

# Wald 통계량 계산
wald_stat_temp = np.dot(np.dot(diff_temp.T, np.linalg.inv(cov_diff_temp)), diff_temp)

# p-value 계산 (자유도는 차이의 차원)
df_temp = len(diff_temp)
p_value_temp = chi2.sf(wald_stat_temp, df_temp)

print(f"\nTemporary Component - Hausman Test p-value: {p_value_temp}")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


고정효과 패널 분석 결과 (Permanent Component):
                           PanelOLS Estimation Summary                           
Dep. Variable:     Permanent_Component   R-squared:                        0.7302
Estimator:                    PanelOLS   R-squared (Between):              0.5440
No. Observations:               291809   R-squared (Within):               0.7302
Date:                 Thu, Oct 24 2024   R-squared (Overall):              0.6127
Time:                         17:04:02   Log-likelihood                -1.709e+06
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   4.937e+04
Entities:                           13   P-value                           0.0000
Avg Obs:                     2.245e+04   Distribution:               F(16,291780)
Min Obs:                     1.017e+04                                           
Max Obs:                     3.318e+04   F-statistic (robust)

## 조절변수 research (size)



In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-89-58ee1529768c>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [ ]:
factor = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/factor.csv')
factor

,Country,User acquisition,Impressions,Est. earnings (KRW)_x,Observed eCPM (KRW),Population,Youth_Support_Rate,TFR,GNI,GDP,...,HOF_PD,HOF_INDIV,HOF_MAS,HOF_UA,HOF_LTO,HOF_INDL,Structure,Kids,Size,segment
0,Argentina,0.220297,0.254660,0.028248,0.024828,0.125993,0.513514,0.507109,0.088108,0.022225,...,0.426966,0.484211,0.518519,0.866667,0.252632,0.639175,1.129956,0.943916,0.314330,3
1,Australia,0.010981,0.016686,0.042084,0.878771,0.068784,0.351351,0.388626,0.659988,0.063282,...,0.303371,0.715789,0.580247,0.477778,0.536842,0.731959,2.520538,0.685410,0.231003,4
2,Austria,0.003328,0.003991,0.012181,0.498275,0.016413,0.189189,0.331754,0.609725,0.015870,...,0.000000,0.757895,0.802469,0.688889,0.442105,0.649485,2.100000,0.485436,0.150000,4
3,Brazil,1.000000,1.000000,0.246172,0.098531,0.633037,0.351351,0.398104,0.047335,0.072918,...,0.651685,0.326316,0.432099,0.755556,0.242105,0.608247,0.932884,0.694415,1.172932,2
4,Canada,0.028610,0.031990,0.063964,0.823867,0.105452,0.243243,0.327014,0.572466,0.081569,...,0.314607,0.705263,0.469136,0.444444,0.515789,0.701031,2.348117,0.529583,0.281110,4
5,Chile,0.239710,0.251707,0.083603,0.143354,0.047980,0.297297,0.355450,0.131927,0.009163,...,0.584270,0.463158,0.172840,0.866667,0.073684,0.701031,1.291158,0.605246,0.114945,3
6,Colombia,0.109790,0.114736,0.014495,0.033918,0.144561,0.405405,0.426540,0.028237,0.010849,...,0.629213,0.252632,0.617284,0.800000,0.010526,0.855670,0.658875,0.770078,0.341997,3
7,Costa Rica,0.019097,0.023944,0.005873,0.077651,0.005390,0.351351,0.345972,0.100410,0.000000,...,0.269663,0.105263,0.086420,0.866667,0.242105,0.391753,0.703141,0.644889,0.012075,3
8,Ecuador,0.043774,0.032838,0.003107,0.020515,0.044191,0.594595,0.563981,0.025893,0.001835,...,0.752809,0.200000,0.604938,0.655556,0.200000,0.515464,0.575103,1.070917,0.097733,3
9,Egypt,0.022349,0.012260,0.000000,0.000000,0.328203,1.000000,0.962085,0.000000,0.016078,...,0.775281,0.084211,0.629630,0.522222,0.178947,0.000000,0.525905,1.813981,0.416094,3


In [ ]:
data = pd.merge(data, factor, on = 'Country', how = 'left')

In [ ]:
data

,Unnamed: 0,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,...,HOF_PD,HOF_INDIV,HOF_MAS,HOF_UA,HOF_LTO,HOF_INDL,Structure,Kids,Size,segment
0,0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,40,1,0.0,0.0,...,0.426966,0.484211,0.518519,0.866667,0.252632,0.639175,1.129956,0.943916,0.314330,3
1,1,2021-07-29,퍼즐놀이,Australia,오세아니아,영어,12,1,0.0,0.0,...,0.303371,0.715789,0.580247,0.477778,0.536842,0.731959,2.520538,0.685410,0.231003,4
2,2,2021-07-29,퍼즐놀이,Austria,유럽,영어,1,0,0.0,0.0,...,0.000000,0.757895,0.802469,0.688889,0.442105,0.649485,2.100000,0.485436,0.150000,4
3,3,2021-07-29,퍼즐놀이,Brazil,중남미,포르투갈어,102,10,0.0,0.0,...,0.651685,0.326316,0.432099,0.755556,0.242105,0.608247,0.932884,0.694415,1.172932,2
4,4,2021-07-29,퍼즐놀이,Canada,북미,영어,21,3,0.0,0.0,...,0.314607,0.705263,0.469136,0.444444,0.515789,0.701031,2.348117,0.529583,0.281110,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291804,291804,2024-08-09,여름휴가,Taiwan,아시아,중국어,23,8,0.0,0.0,...,0.528090,0.368421,0.382716,0.677778,0.863158,0.505155,1.459198,0.248565,0.170540,3
291805,291805,2024-08-09,여름휴가,United Arab Emirates,중동,아랍어,33,23,0.0,818.0,...,0.707865,0.326316,0.469136,0.644444,0.178947,0.226804,1.882711,0.370129,0.068775,4
291806,291806,2024-08-09,여름휴가,United Kingdom,유럽,영어,72,44,0.0,1920.0,...,0.269663,0.747368,0.641975,0.300000,0.578947,0.711340,2.268810,0.652082,0.584327,4
291807,291807,2024-08-09,여름휴가,United States,북미,영어,378,218,793.0,11026.0,...,0.325843,0.578947,0.592593,0.422222,0.473684,0.701031,2.000000,0.688101,2.000000,1


In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_Acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 변수 생성
if 'Language_Game_Flag' not in data.columns:
    data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 새로운 kids 변수가 있다고 가정하고 조절변수 추가
# 예를 들어, 'kids' 변수가 이미 데이터에 존재한다고 가정
# 상호작용 항 추가
data['Language_Game_Flag_Size'] = data['Language_Game_Flag'] * data['Size']
data['Games_Flag_Size'] = data['Games_Flag'] * data['Size']
data['Youtube_Flag_Size'] = data['YouTube_Flag'] * data['Size']

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'similarity_score',
                     'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'day__games', 'Category_ID_youtube',
                    'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day_unsimilar_youtube', 'video_potentiality_unsimilar_youtube', 'YouTube_Flag',
                    'Games_Flag','Language_Game_Flag' ,'Language_Game_Flag_Size','Youtube_Flag_Size','Games_Flag_Size']

# 변수명에서 '+' 기호를 '_'로 교체하여 에러 방지
independent_vars = [var.replace('+', '_') for var in independent_vars]
data.columns = data.columns.str.replace('+', '_')

# VIF 계산을 위한 데이터 준비
X = data[independent_vars].select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Games_Flag와 Language_Game_Flag, Language_Game_Flag_kids를 명시적으로 추가
for flag in ['Games_Flag', 'Language_Game_Flag', 'Language_Game_Flag_Size']:
    if flag not in low_vif_features:
        low_vif_features.append(flag)

# Kalman Filter 설정
user_acquisition = data['User_Acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 날짜, AppName, Country를 패널 분석용 식별자로 사용 (2개의 계층만 사용)
data['Date'] = pd.to_datetime(data['Date'])

# MultiIndex 설정 (AppName과 Date 또는 Country와 Date)
panel_data = data.set_index(['AppName_original', 'Date'])


# 랜덤효과 패널 회귀 분석 (Permanent Component 분석)
model_perm_re = RandomEffects.from_formula('Permanent_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Permanent Component):")
print(model_perm_re)


# 랜덤효과 패널 회귀 분석 (Temporary Component 분석)
model_temp_re = RandomEffects.from_formula('Temporary_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Temporary Component):")
print(model_temp_re)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss



랜덤효과 패널 분석 결과 (Permanent Component):
                         RandomEffects Estimation Summary                        
Dep. Variable:     Permanent_Component   R-squared:                        0.8086
Estimator:               RandomEffects   R-squared (Between):              0.9004
No. Observations:               291809   R-squared (Within):               0.6882
Date:                 Fri, Oct 25 2024   R-squared (Overall):              0.8086
Time:                         05:42:30   Log-likelihood                -1.783e+06
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   6.486e+04
Entities:                           13   P-value                           0.0000
Avg Obs:                     2.245e+04   Distribution:               F(19,291790)
Min Obs:                     1.017e+04                                           
Max Obs:                     3.318e+04   F-statistic (robust

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-103-58ee1529768c>:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [ ]:
factor = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/factor.csv')
data_merge = pd.merge(data, factor, on = 'Country', how = 'left')
data = data_merge.copy()

In [ ]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_Acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 변수 생성
if 'Language_Game_Flag' not in data.columns:
    data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)


data['Language_Game_Flag_Structure'] = data['Language_Game_Flag'] * data['Structure']
data['Games_Flag_Structure'] = data['Games_Flag'] * data['Structure']
data['Youtube_Flag_Structure'] = data['YouTube_Flag'] * data['Structure']

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'similarity_score',
                     'video_potentiality_youtube',
                    'video_potentiality_games', 'day_youtube', 'day__games', 'Category_ID_youtube',
                    'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day_unsimilar_youtube', 'video_potentiality_unsimilar_youtube', 'YouTube_Flag',
                    'Games_Flag', 'Language_Game_Flag','Language_Game_Flag_Structure','Games_Flag_Structure','Youtube_Flag_Structure']

# 변수명에서 '+' 기호를 '_'로 교체하여 에러 방지
independent_vars = [var.replace('+', '_') for var in independent_vars]
data.columns = data.columns.str.replace('+', '_')

# VIF 계산을 위한 데이터 준비
X = data[independent_vars].select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
# vif_data = pd.DataFrame()
# vif_data["Feature"] = X_scaled.columns
# vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# # VIF가 10 이하인 변수들만 선택
# low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Games_Flag와 Language_Game_Flag, Language_Game_Flag_kids를 명시적으로 추가
for flag in ['Games_Flag', 'Language_Game_Flag','Language_Game_Flag_Structure']:
    if flag not in low_vif_features:
        low_vif_features.append(flag)

# Kalman Filter 설정
user_acquisition = data['User_Acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 날짜, AppName, Country를 패널 분석용 식별자로 사용 (2개의 계층만 사용)
data['Date'] = pd.to_datetime(data['Date'])

# MultiIndex 설정 (AppName과 Date 또는 Country와 Date)
panel_data = data.set_index(['AppName_original', 'Date'])


# 랜덤효과 패널 회귀 분석 (Permanent Component 분석)
model_perm_re = RandomEffects.from_formula('Permanent_Component ~ ' + ' + '.join(independent_vars), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Permanent Component):")
print(model_perm_re)


# 랜덤효과 패널 회귀 분석 (Temporary Component 분석)
model_temp_re = RandomEffects.from_formula('Temporary_Component ~ ' + ' + '.join(independent_vars), data=panel_data).fit()

# 랜덤효과 결과 출력
print("\n랜덤효과 패널 분석 결과 (Temporary Component):")
print(model_temp_re)


랜덤효과 패널 분석 결과 (Permanent Component):
                         RandomEffects Estimation Summary                        
Dep. Variable:     Permanent_Component   R-squared:                        0.8076
Estimator:               RandomEffects   R-squared (Between):              0.8997
No. Observations:               291809   R-squared (Within):               0.6870
Date:                 Fri, Oct 25 2024   R-squared (Overall):              0.8076
Time:                         05:54:19   Log-likelihood                -1.784e+06
Cov. Estimator:             Unadjusted                                           
                                         F-statistic:                   6.448e+04
Entities:                           13   P-value                           0.0000
Avg Obs:                     2.245e+04   Distribution:               F(19,291790)
Min Obs:                     1.017e+04                                           
Max Obs:                     3.318e+04   F-statistic (robust